In [1]:
# %matplotlib inline
import numpy as np
import scipy
import pandas
import matplotlib.pyplot as plt
#import statsmodels.formula.api as sm
import seaborn as sns
import sklearn as sl
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')
#pd.set_option('display.float_format', lambda x: '%.3f' % x)

//miniconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
pandas.set_option('display.max_columns', 20)
pandas.set_option('display.width', 350)

Novelty Effect

One of the most common issues data scientists face when dealing with A/B testing is the so-called novelty effect. The problem with novelty effect is the following: when you give users the chance to try a new feature, at first they might try it out just out of curiosity, even if the feature is not actually better. So, say you run a test for 2 weeks, you are actually capturing in those two weeks the phase in which customers are extensively using your feature just because it is new and they are curious, i.e. novelty effect.

You then check your test results and see, for instance, that engagement or time spent on site are way up for test vs control. So you give that feature to all users, but after some time you actually see a steep decline in engagement or time spent. This is a really really common issue especially on sites whose main metrics are related to clicks and time spent on site, like social networks or most businesses that make money via ads.

A little bit more formally, the problem is how to isolate the effect of the new feature vs the effect of novelty, which is unrelated to the new feature and it always happens whenever a user sees something new. And novelty effect is the specific (and most common) example of a much wider topic: how to make sure I am testing only one specific thing and not multiple things at the same time. For instance, let’s say you run a test giving some users a lower price. How to isolate the effect of the lower price vs the excitement of getting a discount?

Note that ironically the opposite also happens. That is, if you give users a new experience, at first they might hate it cause it is not what they are used to and they feel they have to re-learn how to use the product. This is called change aversion. However, in practice, this is a much smaller problem from an A/B testing standpoint because it only affects major product redesigns, which are way rarer than small UI tweaks and they are often not even A/B tested (it is hard to A/B test a major change, like a new logo or a totally new site redesign).

The obvious solution for novelty effect would be to run tests longer, giving test users enough time to get rid of the novelty effect. However, that’s hardly efficient, and the cost of having to run tests for longer would probably outweigh the benefits coming from more reliable results.

Below we will go through an A/B test that was affected by novelty effect and we will see what companies do to make sure that test results are still reliable.

Data comes from a social network and the new product being tested is the first version of a friend recommendation feature. The metric that’s been chosen to evaluate the test is average number of pages visited per user. Given that most social networks monetize via ads and assuming constant the number of ads clicked per # of pages visited, that metric essentially means revenue. The higher that number, the more money the company makes.

In [5]:
data = pandas.read_csv('./dataset/novelty_effect.csv')

In [4]:
data.head()

,user_id,signup_date,test_date,browser,test,pages_visited
0,34,2015-01-01,2015-08-15,Chrome,0,6
1,59,2015-01-01,2015-08-12,Chrome,1,6
2,178,2015-01-01,2015-08-10,Safari,1,3
3,383,2015-01-01,2015-08-05,Firefox,1,9
4,397,2015-01-01,2015-08-27,IE,0,1


In [8]:
data.shape

(97873, 6)

In [6]:
from scipy import stats

In [7]:
#t-test of test vs control for our target metric 
test = stats.ttest_ind(data.loc[data['test'] == 1]['pages_visited'], data.loc[data['test'] == 0]['pages_visited'], equal_var=False)

In [11]:
print(test.statistic)

5.474295518566027


In [12]:
print(test.pvalue)

4.403954129457701e-08


In [13]:
if (test.pvalue>0.05):
  print ("Non-significant results")
elif (test.statistic>0):
  print ("Statistically better results")
else:
  print ("Statistically worse results")

Statistically better results


So, in this case, the test appears to be a clear winner. The increase in page views is close to 2%, which would mean a massive revenue increase for any medium or large company that makes money with ads.

However, this is a classical example of a test that might be affected by novelty effect. After all, people see these new friends suggested to them and they might be prompted at first to click on their profiles just out of curiosity, cause they want to find out how the new feature works. In cases like this, you need to make sure the gain you see is not coming from novelty effect.

There is a key catch about novelty effect: almost by definition this affects mainly returning users. After all, for new users everything is new. So that particular feature cannot have novelty effect by itself. Therefore, the most commonly used way to check for novelty effect is by segmenting users in new vs returning. If the feature is winning for returning users, but not for new users, that’s a really strong sign that novelty effect dynamics are happening.

Btw note that segmenting users by new vs returning is always a useful exercise when running an A/B test. That gives you crucial information on your test. A healthy A/B test should do well on new users. Otherwise, the risk is that you keep optimizing for your old users and end up in a local optimum, not being able to capture opportunities outside of your main user base.

In [14]:
#segment users into new vs old. We define new as those for which the test/control experience was the same as their sign-up date
#now let's do the test for old users and new users separately
  
#old users
ab_test_old = stats.ttest_ind(data.loc[(data['test'] == 1) & (data['signup_date']!=data['test_date'])]['pages_visited'], 
                              data.loc[(data['test'] == 0) & (data['signup_date']!=data['test_date'])]['pages_visited'], 
                              equal_var=False)
#t statistics

In [15]:
#p-value
print(ab_test_old.pvalue)

4.859481805141211e-12


In [17]:
if (ab_test_old.pvalue>0.05/2):
    print ("Returning users: Non-significant results")
elif (ab_test_old.statistic>0):
    print ("Returning users: Statistically better results")
else:
    print ("Returning users: Statistically worse results")

Returning users: Statistically better results


In [18]:
#new users
ab_test_new = stats.ttest_ind(data.loc[(data['test'] == 1) & (data['signup_date']==data['test_date'])]['pages_visited'], 
                              data.loc[(data['test'] == 0) & (data['signup_date']==data['test_date'])]['pages_visited'], 
                              equal_var=False)
#t statistics
print(ab_test_new.statistic)

-1.0809363577979878


In [21]:
print(ab_test_new.pvalue)

0.27973874896130424


In [20]:
if (ab_test_new.pvalue>0.05/2):
    print ("New users: Non-significant results")
elif (ab_test_new.statistic>0):
    print ("New users: Statistically better results")
else:
    print ("New users: Statistically worse results")

New users: Non-significant results


There you go! Our test was winning because old users are using the new feature a lot. But the new feature has no impact on the new user behavior. In practice, this is a really strong sign of novelty effect.

Btw this exercise was just about showing how to find novelty effect. But obviously everything that we learned previously should be applied to these two new tests as well. Like you should define in advance minimum effect size for both groups and make sure you have enough people per group, you should make sure randomization worked fine, etc. Essentially, everything should be duplicated for the new user test as well as for the old user test.

If you want to check statistically significance on segments of your population, you need to define those segments in advance, before running the test. And make sure each test on each subset of the user base is statistically sound.